In [61]:

"""
This is an upgraded version of Ceshine's LGBM starter script, simply adding
more average features and weekly average features on it.

Run 1 Store validation  : .py 1s 045
Run 1 Store submission  : .py 1ss 045
Run all Store validation: .py val 045
Run all Store submission: .py a 045


"""
from datetime import date, timedelta

import pandas as pd
import numpy as np
import lightgbm as lgb
import sys
import math
import gc
import sklearn.metrics as skl_metrics
from sklearn.metrics import mean_squared_error

from nwrmsle_eval import eval_test

# import math
# import sklearn.metrics as skl_metrics
# from sklearn.metrics import mean_squared_error

from logging import StreamHandler, DEBUG, Formatter, FileHandler, getLogger

logger = getLogger(__name__)

pd.options.mode.chained_assignment = None  # default='warn'

DIR = '../logs/'

log_fmt = Formatter('%(asctime)s %(name)s %(lineno)d [%(levelname)s]\
    [%(funcName)s] %(message)s ')
handler = StreamHandler()
handler.setLevel('INFO')
handler.setFormatter(log_fmt)
logger.addHandler(handler)

handler = FileHandler(DIR + 'train.py.log', 'a')
handler.setLevel(DEBUG)
handler.setFormatter(log_fmt)
logger.setLevel(DEBUG)
logger.addHandler(handler)

logger.info('start')


##########################################################################
# 1s : validate one store data
# 1ss: submit one store
# val: validate all data
# No parameter: submit all data 

param_1 = "1s"
param_2 = "tmp"
    

print("input parameter = ", param_1)
print("Test/val number = ", param_2)
submit_filename = '../submit/T' + param_2 + '.csv.gz'
val_filename = '../data/V' + param_2 + '.p'

logger.info(submit_filename)
logger.info(val_filename)

if  ((param_1 == "1ss") or (param_1 == "1s")):
    train_out = pd.read_pickle('../data/storeitem_train_1s.p')
    val_out = pd.read_pickle('../data/storeitem_val_1s.p')
    X_test_out = pd.read_pickle('../data/storeitem_test_1s.p')

    item_train_out = pd.read_pickle('../data/item_train_1s.p')
    item_val_out = pd.read_pickle('../data/item_val_1s.p')
    item_X_test_out = pd.read_pickle('../data/item_test_1s.p')

    store_train_out = pd.read_pickle('../data/store_train_1s.p')
    store_val_out = pd.read_pickle('../data/store_val_1s.p')
    store_X_test_out = pd.read_pickle('../data/store_test_1s.p')

    s_f_train_out = pd.read_pickle('../data/storefamily_train_1s.p')
    s_f_val_out = pd.read_pickle('../data/storefamily_val_1s.p')
    s_f_X_test_out = pd.read_pickle('../data/storefamily_test_1s.p')
    
    df_test = pd.read_csv(
        "../input/test_1s.csv", usecols=[0, 1, 2, 3, 4],
        dtype={'onpromotion': bool},
        parse_dates=["date"]  # , date_parser=parser
    ).set_index(
        ['store_nbr', 'item_nbr', 'date']
    )
    
else:
    train_out = pd.read_pickle('../data/storeitem_train.p')
    val_out = pd.read_pickle('../data/storeitem_val.p')
    X_test_out = pd.read_pickle('../data/storeitem_test.p')

    item_train_out = pd.read_pickle('../data/item_train.p')
    item_val_out = pd.read_pickle('../data/item_val.p')
    item_X_test_out = pd.read_pickle('../data/item_test.p')

    store_train_out = pd.read_pickle('../data/store_train.p')
    store_val_out = pd.read_pickle('../data/store_val.p')
    store_X_test_out = pd.read_pickle('../data/store_test.p')

    s_f_train_out = pd.read_pickle('../data/storefamily_train.p')
    s_f_val_out = pd.read_pickle('../data/storefamily_val.p')
    s_f_X_test_out = pd.read_pickle('../data/storefamily_test.p')

    df_test = pd.read_csv(
        "../input/test.csv", usecols=[0, 1, 2, 3, 4],
        dtype={'onpromotion': bool},
        parse_dates=["date"]  # , date_parser=parser
    ).set_index(
        ['store_nbr', 'item_nbr', 'date']
    )


# On validation step, need remove last 2 weeks in the train data
# train_out["date"] = pd.to_datetime(train_out["date"])
#if ((param_1 == "val") or (param_1 == "1s")):
#    train_out = train_out.loc[train_out["date"] < '2017-07-19', ]

  
items = pd.read_csv(
    "../input/items.csv",
)

items_val = pd.read_csv(
    "../input/items.csv",
).set_index("item_nbr")
items_val = items_val.reindex(val_out['item_nbr'])

logger.info('Load data successful')


2018-01-05 13:28:53,878 __main__ 50 [INFO]    [<module>] start 
2018-01-05 13:28:53,878 __main__ 50 [INFO]    [<module>] start 
2018-01-05 13:28:53,878 __main__ 50 [INFO]    [<module>] start 
2018-01-05 13:28:53,878 __main__ 50 [INFO]    [<module>] start 
2018-01-05 13:28:53,878 __main__ 50 [INFO]    [<module>] start 
2018-01-05 13:28:53,878 __main__ 50 [INFO]    [<module>] start 
2018-01-05 13:28:53,878 __main__ 50 [INFO]    [<module>] start 
2018-01-05 13:28:53,878 __main__ 50 [INFO]    [<module>] start 
2018-01-05 13:28:53,878 __main__ 50 [INFO]    [<module>] start 
2018-01-05 13:28:53,878 __main__ 50 [INFO]    [<module>] start 
2018-01-05 13:28:53,878 __main__ 50 [INFO]    [<module>] start 
2018-01-05 13:28:53,878 __main__ 50 [INFO]    [<module>] start 
2018-01-05 13:28:53,878 __main__ 50 [INFO]    [<module>] start 
2018-01-05 13:28:53,894 __main__ 68 [INFO]    [<module>] ../submit/Ttmp.csv.gz 
2018-01-05 13:28:53,894 __main__ 68 [INFO]    [<module>] ../submit/Ttmp.csv.gz 
2018-01-

input parameter =  1s
Test/val number =  tmp


2018-01-05 13:28:54,193 __main__ 137 [INFO]    [<module>] Load data successful 
2018-01-05 13:28:54,193 __main__ 137 [INFO]    [<module>] Load data successful 
2018-01-05 13:28:54,193 __main__ 137 [INFO]    [<module>] Load data successful 
2018-01-05 13:28:54,193 __main__ 137 [INFO]    [<module>] Load data successful 
2018-01-05 13:28:54,193 __main__ 137 [INFO]    [<module>] Load data successful 
2018-01-05 13:28:54,193 __main__ 137 [INFO]    [<module>] Load data successful 
2018-01-05 13:28:54,193 __main__ 137 [INFO]    [<module>] Load data successful 
2018-01-05 13:28:54,193 __main__ 137 [INFO]    [<module>] Load data successful 
2018-01-05 13:28:54,193 __main__ 137 [INFO]    [<module>] Load data successful 
2018-01-05 13:28:54,193 __main__ 137 [INFO]    [<module>] Load data successful 
2018-01-05 13:28:54,193 __main__ 137 [INFO]    [<module>] Load data successful 
2018-01-05 13:28:54,193 __main__ 137 [INFO]    [<module>] Load data successful 
2018-01-05 13:28:54,193 __main__ 137 [IN

In [62]:
train_out.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46241 entries, 0 to 46240
Columns: 144 entries, index to day16
dtypes: float64(121), int64(6), object(1), uint8(16)
memory usage: 45.9+ MB


In [63]:
train_out.shape

(46241, 144)

In [72]:
items.groupby(['family', 'class']).size()

family                      class
AUTOMOTIVE                  6806       44
                            6808       22
                            6810       55
                            6848       44
BEAUTY                      4210       11
                            4212       11
                            4214       22
                            4222       22
                            4228       11
                            4250       11
                            4252       44
                            4254       44
BEVERAGES                   1114      429
                            1115       22
                            1116      165
                            1118      253
                            1120      649
                            1122      869
                            1124     1067
                            1126      396
                            1132      231
                            1136      297
                            1138       88


In [64]:
#train_out["date"] = pd.to_datetime(train_out["date"])
if ((param_1 == "val") or (param_1 == "1s")):
    train_out = train_out.loc[train_out["date"] < date(2017, 7, 19), ]


In [65]:
train_out.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 39127 entries, 0 to 39126
Columns: 144 entries, index to day16
dtypes: float64(121), int64(6), object(1), uint8(16)
memory usage: 39.1+ MB


In [66]:
train_out.head(1)

,index,date,day_1_2017,item_nbr,mean_140_2017,mean_14_2017,mean_182_2017,mean_21_2017,mean_30_2017,mean_364_2017,...,day7,day8,day9,day10,day11,day12,day13,day14,day15,day16
0,0,2016-08-03,0.0,96995,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [42]:
train_out.shape

(39127, 144)

In [31]:
print(train_out.groupby(['date']).size())

date
2016-08-03    3557
2016-08-10    3557
2016-08-17    3557
2016-08-24    3557
2017-05-31    3557
2017-06-07    3557
2017-06-14    3557
2017-06-21    3557
2017-06-28    3557
2017-07-05    3557
2017-07-12    3557
dtype: int64


In [67]:

###############################################################################
# Delete index columns before merge 
del train_out["index"]
del item_train_out["index"]
del store_train_out["index"]
del s_f_train_out["index"]

# Merge store-family features
items_c = items.copy()
del items_c["class"], items_c["perishable"]


s_f_train_out = pd.merge(s_f_train_out, items_c, how = 'inner', on=['family'] )
s_f_val_out = pd.merge(s_f_val_out, items_c, how = 'inner', on=['family'] )
s_f_X_test_out = pd.merge(s_f_X_test_out, items_c, how = 'inner', on = ['family'] )

del s_f_train_out['family'], s_f_val_out['family'], s_f_X_test_out['family']

train_out = pd.merge(train_out, s_f_train_out, how='inner', on=['store_nbr','item_nbr','date'])
val_out = pd.merge(val_out, s_f_val_out, how='inner', on=['store_nbr','item_nbr','date'])
X_test_out = pd.merge(X_test_out, s_f_X_test_out, how='inner', on=['store_nbr','item_nbr','date'])

del items_c,s_f_train_out, s_f_val_out, s_f_X_test_out
gc.collect()



# Merge item features 
train_out = pd.merge(train_out, item_train_out, how='inner', on=['item_nbr','date'])
val_out = pd.merge(val_out, item_val_out, how='inner', on=['item_nbr','date'])
X_test_out = pd.merge(X_test_out, item_X_test_out, how='inner', on=['item_nbr','date'])

del item_train_out, item_val_out, item_X_test_out
gc.collect()



# Merge store features
train_out = pd.merge(train_out, store_train_out, how='inner', on=['store_nbr','date'])
val_out = pd.merge(val_out, store_val_out, how='inner', on=['store_nbr','date'])
X_test_out = pd.merge(X_test_out, store_X_test_out, how='inner', on=['store_nbr','date'])

print(train_out.groupby(['date']).size())

del store_train_out, store_val_out, store_X_test_out
gc.collect()


###############################################################################
logger.info('Preparing traing dataset...')
    

all_columns = train_out.columns.tolist()

y_columns = ['day'+str(i) for i in range(1, 17)]
x_columns = [item for item in all_columns if item not in y_columns]

features_all = x_columns
features_all.remove("date") 
features_all.remove("item_nbr") 
features_all.remove("store_nbr") 

X_train_out = train_out[x_columns]
X_val_out = val_out[x_columns]

y_train = train_out[y_columns].values
y_val = val_out[y_columns].values

X_train_allF = X_train_out[features_all]
X_val_allF = X_val_out[features_all]
X_test_allF = X_test_out[features_all]

items = items.set_index("item_nbr")
items = items.reindex(train_out.item_nbr)

del train_out
del X_train_out, X_val_out
gc.collect()

2018-01-05 13:29:16,854 __main__ 51 [INFO]    [<module>] Preparing traing dataset... 
2018-01-05 13:29:16,854 __main__ 51 [INFO]    [<module>] Preparing traing dataset... 
2018-01-05 13:29:16,854 __main__ 51 [INFO]    [<module>] Preparing traing dataset... 
2018-01-05 13:29:16,854 __main__ 51 [INFO]    [<module>] Preparing traing dataset... 
2018-01-05 13:29:16,854 __main__ 51 [INFO]    [<module>] Preparing traing dataset... 
2018-01-05 13:29:16,854 __main__ 51 [INFO]    [<module>] Preparing traing dataset... 
2018-01-05 13:29:16,854 __main__ 51 [INFO]    [<module>] Preparing traing dataset... 
2018-01-05 13:29:16,854 __main__ 51 [INFO]    [<module>] Preparing traing dataset... 
2018-01-05 13:29:16,854 __main__ 51 [INFO]    [<module>] Preparing traing dataset... 
2018-01-05 13:29:16,854 __main__ 51 [INFO]    [<module>] Preparing traing dataset... 
2018-01-05 13:29:16,854 __main__ 51 [INFO]    [<module>] Preparing traing dataset... 
2018-01-05 13:29:16,854 __main__ 51 [INFO]    [<module

date
2016-08-03    3557
2016-08-10    3557
2016-08-17    3557
2016-08-24    3557
2017-05-31    3557
2017-06-07    3557
2017-06-14    3557
2017-06-21    3557
2017-06-28    3557
2017-07-05    3557
2017-07-12    3557
dtype: int64


117

In [68]:

##########################################################################
logger.info('Training and predicting models...')

params = {
    'num_leaves': 31,
    'objective': 'regression',
    'min_data_in_leaf': 300,
    'learning_rate': 0.1,
    'feature_fraction': 0.8,
    'bagging_fraction': 0.8,
    'bagging_freq': 2,
    'metric': 'l2',
    'num_threads': 4
}

MAX_ROUNDS = 500
val_pred = []
test_pred = []
cate_vars = []


train_week_2017 = 7
if param_1 != "val":
    train_week_2017 = 9
    
features_all = X_train_allF.columns.tolist()

for i in range(16):
    print("=" * 70)
    logger.info("Step %d" % (i+1))
    print("=" * 70)
    features_t = features_all.copy()

    for j in range(16):
        if j != i:
            features_t.remove('ly_1d_d{}'.format(j))
            features_t.remove('l2y_1d_d{}'.format(j))

    for j in range(7):
        if j != i%7:
            features_t.remove('dow_1_{}_mean'.format(j))
            features_t.remove('dow_4_{}_mean'.format(j))
            features_t.remove('dow_8_{}_mean'.format(j))
            features_t.remove('dow_13_{}_mean'.format(j))
            features_t.remove('dow_26_{}_mean'.format(j))
            features_t.remove('dow_52_{}_mean'.format(j))
            features_t.remove('dow_ly3w_{}_mean'.format(j))
            features_t.remove('dow_ly8w_{}_mean'.format(j))
            
            features_t.remove('item_dow_4_{}_mean'.format(j))
            features_t.remove('item_dow_13_{}_mean'.format(j))
            features_t.remove('item_dow_26_{}_mean'.format(j))
            features_t.remove('item_dow_52_{}_mean'.format(j))

            features_t.remove('store_dow_4_{}_mean'.format(j))
            features_t.remove('store_dow_13_{}_mean'.format(j))
            features_t.remove('store_dow_26_{}_mean'.format(j))
            features_t.remove('store_dow_52_{}_mean'.format(j))     

            
            features_t.remove('s_f_dow_4_{}_mean'.format(j))
            features_t.remove('s_f_dow_13_{}_mean'.format(j))
            features_t.remove('s_f_dow_26_{}_mean'.format(j))
            features_t.remove('s_f_dow_52_{}_mean'.format(j))
            
    X_train = X_train_allF[features_t]
    X_val = X_val_allF[features_t]
    X_test = X_test_allF[features_t]
	
     
    dtrain = lgb.Dataset(
        X_train, label=y_train[:, i],
        categorical_feature=cate_vars,
        weight=pd.concat([items["perishable"]]) * 0.25 + 1
    )

    if ((param_1 == "val") or (param_1 == "1s")):
        dval = lgb.Dataset(
             X_val, label=y_val[:, i], reference=dtrain,
             weight=items_val["perishable"] * 0.25 + 1,
             categorical_feature=cate_vars)

        bst = lgb.train(
            params, dtrain, num_boost_round=MAX_ROUNDS,
            valid_sets=[dtrain, dval], early_stopping_rounds=50,
            verbose_eval=100
        )

    else:

        bst = lgb.train(
            params, dtrain, num_boost_round=MAX_ROUNDS,
            valid_sets=[dtrain], early_stopping_rounds=50, verbose_eval=100
        )

    logger.info("\n".join(("%s: %.2f" % x) for x in sorted(
        zip(X_train.columns, bst.feature_importance("gain")),
        key=lambda x: x[1], reverse=True
    )))

    test_pred.append(bst.predict(
        X_test, num_iteration=bst.best_iteration or MAX_ROUNDS))

    if ((param_1 == "val") or (param_1 == "1s")):
        val_pred.append(bst.predict(
            X_val, num_iteration=bst.best_iteration or MAX_ROUNDS))

del X_train, y_train
del dtrain
gc.collect()


2018-01-05 13:29:21,319 __main__ 3 [INFO]    [<module>] Training and predicting models... 
2018-01-05 13:29:21,319 __main__ 3 [INFO]    [<module>] Training and predicting models... 
2018-01-05 13:29:21,319 __main__ 3 [INFO]    [<module>] Training and predicting models... 
2018-01-05 13:29:21,319 __main__ 3 [INFO]    [<module>] Training and predicting models... 
2018-01-05 13:29:21,319 __main__ 3 [INFO]    [<module>] Training and predicting models... 
2018-01-05 13:29:21,319 __main__ 3 [INFO]    [<module>] Training and predicting models... 
2018-01-05 13:29:21,319 __main__ 3 [INFO]    [<module>] Training and predicting models... 
2018-01-05 13:29:21,319 __main__ 3 [INFO]    [<module>] Training and predicting models... 
2018-01-05 13:29:21,319 __main__ 3 [INFO]    [<module>] Training and predicting models... 
2018-01-05 13:29:21,319 __main__ 3 [INFO]    [<module>] Training and predicting models... 
2018-01-05 13:29:21,319 __main__ 3 [INFO]    [<module>] Training and predicting models... 

Training until validation scores don't improve for 50 rounds.
[100]	training's l2: 0.244592	valid_1's l2: 0.268021


2018-01-05 13:29:24,052 __main__ 99 [INFO]    [<module>] mean_14_2017: 56911.41
mean_7_2017: 33038.81
mean_21_2017: 18838.60
dow_26_0_mean: 4022.11
promo_0: 3607.30
mean_3_2017: 2834.76
dow_8_0_mean: 2309.26
dow_13_0_mean: 1921.10
dow_52_0_mean: 1605.98
mean_91_2017: 1494.84
day_1_2017: 829.20
mean_42_2017: 722.85
promo_14_2017: 593.89
dow_4_0_mean: 550.13
mean_30_2017: 406.46
mean_60_2017: 404.70
dow_1_0_mean: 399.97
mean_182_2017: 353.22
mean_140_2017: 339.25
item_mean_7_2017: 301.89
dow_ly8w_0_mean: 274.28
promo_140_2017: 252.42
promo_60_2017: 229.79
s_f_mean_182_2017: 195.79
item_dow_4_0_mean: 172.64
mean_ly_n16d_2017: 164.49
item_day_1_2017: 156.91
s_f_dow_4_0_mean: 154.88
item_mean_21_2017: 140.49
mean_ly_30_2017: 140.21
s_f_dow_13_0_mean: 138.43
dow_ly3w_0_mean: 135.12
promo_7: 116.93
item_dow_52_0_mean: 113.69
item_dow_13_0_mean: 112.78
ly_1d_d0: 111.96
item_mean_182_2017: 108.39
s_f_mean_21_2017: 108.30
s_f_day_1_2017: 100.16
l2y_1d_d0: 95.32
store_day_1_2017: 93.38
s_f_mean_7

2018-01-05 13:29:24,052 __main__ 99 [INFO]    [<module>] mean_14_2017: 56911.41
mean_7_2017: 33038.81
mean_21_2017: 18838.60
dow_26_0_mean: 4022.11
promo_0: 3607.30
mean_3_2017: 2834.76
dow_8_0_mean: 2309.26
dow_13_0_mean: 1921.10
dow_52_0_mean: 1605.98
mean_91_2017: 1494.84
day_1_2017: 829.20
mean_42_2017: 722.85
promo_14_2017: 593.89
dow_4_0_mean: 550.13
mean_30_2017: 406.46
mean_60_2017: 404.70
dow_1_0_mean: 399.97
mean_182_2017: 353.22
mean_140_2017: 339.25
item_mean_7_2017: 301.89
dow_ly8w_0_mean: 274.28
promo_140_2017: 252.42
promo_60_2017: 229.79
s_f_mean_182_2017: 195.79
item_dow_4_0_mean: 172.64
mean_ly_n16d_2017: 164.49
item_day_1_2017: 156.91
s_f_dow_4_0_mean: 154.88
item_mean_21_2017: 140.49
mean_ly_30_2017: 140.21
s_f_dow_13_0_mean: 138.43
dow_ly3w_0_mean: 135.12
promo_7: 116.93
item_dow_52_0_mean: 113.69
item_dow_13_0_mean: 112.78
ly_1d_d0: 111.96
item_mean_182_2017: 108.39
s_f_mean_21_2017: 108.30
s_f_day_1_2017: 100.16
l2y_1d_d0: 95.32
store_day_1_2017: 93.38
s_f_mean_7

2018-01-05 13:29:24,052 __main__ 99 [INFO]    [<module>] mean_14_2017: 56911.41
mean_7_2017: 33038.81
mean_21_2017: 18838.60
dow_26_0_mean: 4022.11
promo_0: 3607.30
mean_3_2017: 2834.76
dow_8_0_mean: 2309.26
dow_13_0_mean: 1921.10
dow_52_0_mean: 1605.98
mean_91_2017: 1494.84
day_1_2017: 829.20
mean_42_2017: 722.85
promo_14_2017: 593.89
dow_4_0_mean: 550.13
mean_30_2017: 406.46
mean_60_2017: 404.70
dow_1_0_mean: 399.97
mean_182_2017: 353.22
mean_140_2017: 339.25
item_mean_7_2017: 301.89
dow_ly8w_0_mean: 274.28
promo_140_2017: 252.42
promo_60_2017: 229.79
s_f_mean_182_2017: 195.79
item_dow_4_0_mean: 172.64
mean_ly_n16d_2017: 164.49
item_day_1_2017: 156.91
s_f_dow_4_0_mean: 154.88
item_mean_21_2017: 140.49
mean_ly_30_2017: 140.21
s_f_dow_13_0_mean: 138.43
dow_ly3w_0_mean: 135.12
promo_7: 116.93
item_dow_52_0_mean: 113.69
item_dow_13_0_mean: 112.78
ly_1d_d0: 111.96
item_mean_182_2017: 108.39
s_f_mean_21_2017: 108.30
s_f_day_1_2017: 100.16
l2y_1d_d0: 95.32
store_day_1_2017: 93.38
s_f_mean_7

...

2018-01-05 13:29:28,431 __main__ 99 [INFO]    [<module>] mean_14_2017: 29820.80
mean_7_2017: 25112.66
mean_21_2017: 15608.85
mean_30_2017: 5888.47
mean_60_2017: 4938.63
dow_8_1_mean: 3192.47
mean_42_2017: 2344.03
dow_13_1_mean: 2086.65
promo_1: 1956.43
dow_26_1_mean: 1545.55
dow_52_1_mean: 1111.20
mean_3_2017: 913.78
dow_4_1_mean: 676.22
item_mean_7_2017: 385.55
promo_14_2017: 374.13
item_dow_4_1_mean: 281.62
day_1_2017: 274.82
dow_1_1_mean: 228.97
dow_ly8w_1_mean: 225.76
mean_140_2017: 209.71
promo_60_2017: 183.03
promo_140_2017: 168.72
mean_91_2017: 163.46
store_day_1_2017: 150.01
dow_ly3w_1_mean: 137.85
mean_182_2017: 133.02
l2y_1d_d1: 115.61
item_mean_91_2017: 98.96
mean_ly_n16d_2017: 97.09
mean_ly_14_2017: 91.43
item_day_1_2017: 89.51
s_f_dow_4_1_mean: 89.02
item_mean_364_2017: 88.95
s_f_day_1_2017: 88.26
promo_7: 86.98
item_mean_182_2017: 86.73
item_mean_21_2017: 85.30
store_mean_182_2017: 83.42
promo_5: 81.45
mean_364_2017: 76.91
s_f_mean_182_2017: 72.60
item_dow_13_1_mean: 72.2

2018-01-05 13:29:28,431 __main__ 99 [INFO]    [<module>] mean_14_2017: 29820.80
mean_7_2017: 25112.66
mean_21_2017: 15608.85
mean_30_2017: 5888.47
mean_60_2017: 4938.63
dow_8_1_mean: 3192.47
mean_42_2017: 2344.03
dow_13_1_mean: 2086.65
promo_1: 1956.43
dow_26_1_mean: 1545.55
dow_52_1_mean: 1111.20
mean_3_2017: 913.78
dow_4_1_mean: 676.22
item_mean_7_2017: 385.55
promo_14_2017: 374.13
item_dow_4_1_mean: 281.62
day_1_2017: 274.82
dow_1_1_mean: 228.97
dow_ly8w_1_mean: 225.76
mean_140_2017: 209.71
promo_60_2017: 183.03
promo_140_2017: 168.72
mean_91_2017: 163.46
store_day_1_2017: 150.01
dow_ly3w_1_mean: 137.85
mean_182_2017: 133.02
l2y_1d_d1: 115.61
item_mean_91_2017: 98.96
mean_ly_n16d_2017: 97.09
mean_ly_14_2017: 91.43
item_day_1_2017: 89.51
s_f_dow_4_1_mean: 89.02
item_mean_364_2017: 88.95
s_f_day_1_2017: 88.26
promo_7: 86.98
item_mean_182_2017: 86.73
item_mean_21_2017: 85.30
store_mean_182_2017: 83.42
promo_5: 81.45
mean_364_2017: 76.91
s_f_mean_182_2017: 72.60
item_dow_13_1_mean: 72.2

2018-01-05 13:29:28,431 __main__ 99 [INFO]    [<module>] mean_14_2017: 29820.80
mean_7_2017: 25112.66
mean_21_2017: 15608.85
mean_30_2017: 5888.47
mean_60_2017: 4938.63
dow_8_1_mean: 3192.47
mean_42_2017: 2344.03
dow_13_1_mean: 2086.65
promo_1: 1956.43
dow_26_1_mean: 1545.55
dow_52_1_mean: 1111.20
mean_3_2017: 913.78
dow_4_1_mean: 676.22
item_mean_7_2017: 385.55
promo_14_2017: 374.13
item_dow_4_1_mean: 281.62
day_1_2017: 274.82
dow_1_1_mean: 228.97
dow_ly8w_1_mean: 225.76
mean_140_2017: 209.71
promo_60_2017: 183.03
promo_140_2017: 168.72
mean_91_2017: 163.46
store_day_1_2017: 150.01
dow_ly3w_1_mean: 137.85
mean_182_2017: 133.02
l2y_1d_d1: 115.61
item_mean_91_2017: 98.96
mean_ly_n16d_2017: 97.09
mean_ly_14_2017: 91.43
item_day_1_2017: 89.51
s_f_dow_4_1_mean: 89.02
item_mean_364_2017: 88.95
s_f_day_1_2017: 88.26
promo_7: 86.98
item_mean_182_2017: 86.73
item_mean_21_2017: 85.30
store_mean_182_2017: 83.42
promo_5: 81.45
mean_364_2017: 76.91
s_f_mean_182_2017: 72.60
item_dow_13_1_mean: 72.2

Early stopping, best iteration is:
[62]	training's l2: 0.273159	valid_1's l2: 0.291598


KeyboardInterrupt: 